# In this notebook we use Pre-trained model which is specified as copy model and we train it on our own annotated images. This strategy helps when your own training data is limited 

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

Mounted at /content/drive
TensorFlow 1.x selected.


In [2]:
!pip install tiffile
!pip install elasticdeform
!pip install keras==2.2.5
!pip install csbdeep
!pip install stardist
!pip install gputools
!pip install btrack

  Created wheel for elasticdeform: filename=elasticdeform-0.4.7-cp36-cp36m-linux_x86_64.whl size=72524 sha256=2847824e246582cc77851a26465979f05abfb2744e4ee741d8ce520a5083d435
  Stored in directory: /root/.cache/pip/wheels/1d/5e/8a/890fbf14dc7f26d5da56968248eb7b85fd7e72870462e2c3e3
Successfully built elasticdeform
     |████████████████████████████████| 337kB 14.0MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 378kB 13.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 419kB 12.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for stardist: filename=stardist-0.6.1-cp36-cp36m-linux_x86_64.whl size=1859809 sha256=de1

In [5]:
%cd '/content/drive/My Drive/SmartSeeds/'
import os
import glob
import sys
sys.path.append('../')
import glob
from Utils import SmartSeeds2D
from tifffile import imread, imwrite
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

/content/drive/My Drive/SmartSeeds


# In the cell below specify the following:

1) Directory where the training data is, inside this directory there should be the two subfolders called Raw and Mask. Inside the Raw folder are the raw images and inside the Mask folder are the labelled images.

2) The training data for doing UNET training is stored in NPZ format so please specify the NPZ filename which is suitable for your data.

3) Model directory is where the trained Neural network models are stored, please chooose a location if you want to change the default location which is where the training data is.

4) Copy Model name is optional, in case you have a previouis trained model and want to re-train it on new data but store it with a new name.

5) Model name is the unique name of the trained models.

In [6]:
Data_dir = '/content/drive/My Drive/FMIChallenge/'
NPZ_filename = 'FMINuclei'
Model_dir = Data_dir
Model_Name = 'FMINuclei'


# In this cell choose the network training parameters for the Neural Network

1) NetworkDepth = Depth of the network, with each increasing depth the image is downsampled by 2 hence the XY dimension of the data / 2^depth has to be greater than 1.

2) Epochs, training for longer epochs ensures a well converged network and requires longer GPU runtimes.

3) Learning rate is the parameter which controls the step size used in the optimization process and it should not be greater than 0.001 at the start of the training.

4) batch size controls the number of images used for doing stochastic gradient descent and is a parameter that is limited by the GPU RAM available, if you do not have a lot of ran batch size < 10 should be optimal. 

5) PatchX,Y is the patch size used for making patches out of the iamge data. The original image is broken down into patches for training. Patch size is chosen based on having enough context for the network to learn but at the same time not being too big to obscure memory usage.

6) Kernel is the receptive field of the neural network, usual choices are 3,5 or 7 but not larger than that. This is the size of the convolutional kernel used in the network

7) n_patches_per_image is the number of patches sampled for each image to create the npz file, choose an optimal value so that the file is not too big for the computer memory. 

8) Rays is the number of rays used the learn the distance map, low rays decreases the spatial resoultion and high rays are able to resolve the shape better.


9) OpenCL is a boolean parameter that is set true if you want to do some opencl computations on the GPU, this requires GPU tools but if you do not have them set this to false.

Some optimal values have been chosen by default and should work well for any NVDIA enabled GPU computer

In [9]:
#Network training parameters
NetworkDepth =4
Epochs = 200
LearningRate = 1.0E-4
batch_size = 1
PatchX = 256
PatchY = 256
Kernel = 3
DownsampleFactor = 1
n_patches_per_image = 16
Rays = 128
use_gpu_opencl = False
GenerateNPZ = False
TrainUNET = True
TrainSTAR = True
grid = (2,2)

In [10]:

model = SmartSeeds2D(BaseDir = Data_dir, NPZfilename = NPZ_filename, model_name = Model_Name, model_dir = Model_dir, n_patches_per_image = n_patches_per_image, DownsampleFactor = DownsampleFactor, GenerateNPZ = GenerateNPZ,TrainUNET = TrainUNET, TrainSTAR = TrainSTAR, grid = grid, PatchX= PatchX, PatchY= PatchY,  use_gpu = use_gpu_opencl,  batch_size = batch_size, depth = NetworkDepth, kern_size = Kernel, n_rays = Rays, epochs = Epochs, learning_rate = LearningRate)

Instance segmentation masks: 12
Semantic segmentation masks: 12
Training StarDistModel model


100%|██████████| 12/12 [00:00<00:00, 199.55it/s]

12
number of images:  12
- training:        10
- validation:       2
Configuration for a :class:`StarDist2D` model.

    Parameters
    ----------
    axes : str or None
        Axes of the input images.
    n_rays : int
        Number of radial directions for the star-convex polygon.
        Recommended to use a power of 2 (default: 32).
    n_channel_in : int
        Number of channels of given input image (default: 1).
    grid : (int,int)
        Subsampling factors (must be powers of 2) for each of the axes.
        Model will predict on a subsampled grid for increased efficiency and larger field of view.
    backbone : str
        Name of the neural network architecture to be used as backbone.
    kwargs : dict
        Overwrite (or add) configuration attributes (see below).


    Attributes
    ----------
    unet_n_depth : int
        Number of U-Net resolution levels (down/up-sampling layers).
    unet_kernel_size : (int,int)
        Convolution kernel size for all (U-Net) con


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Epoch 1/200
  4/100 [>.............................] - ETA: 4:42 - loss: 2.0173 - prob_loss: 0.6916 - dist_loss: 26.5134 - prob_kld: 0.4026 - dist_relevant_mae: 26.5134 - dist_relevant_mse: 952.2837

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.169132). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


...

100/100 [==============================] - 175s 2s/step - loss: 0.3723 - prob_loss: 0.2957 - dist_loss: 1.5307 - prob_kld: 0.0050 - dist_relevant_mae: 1.5293 - dist_relevant_mse: 11.5071 - val_loss: 0.3937 - val_prob_loss: 0.3052 - val_dist_loss: 1.7704 - val_prob_kld: 0.0078 - val_dist_relevant_mae: 1.7690 - val_dist_relevant_mse: 12.9190
Epoch 180/200
100/100 [==============================] - 176s 2s/step - loss: 0.3703 - prob_loss: 0.2952 - dist_loss: 1.5028 - prob_kld: 0.0043 - dist_relevant_mae: 1.5014 - dist_relevant_mse: 11.0618 - val_loss: 0.3942 - val_prob_loss: 0.3086 - val_dist_loss: 1.7127 - val_prob_kld: 0.0112 - val_dist_relevant_mae: 1.7113 - val_dist_relevant_mse: 12.5467
Epoch 181/200
100/100 [==============================] - 174s 2s/step - loss: 0.3689 - prob_loss: 0.2949 - dist_loss: 1.4813 - prob_kld: 0.0052 - dist_relevant_mae: 1.4799 - dist_relevant_mse: 11.0343 - val_loss: 0.3965 - val_prob_loss: 0.3079 - val_dist_loss: 1.7724 - val_prob_kld: 0.0105 - val_dist_

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.187090). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


100/100 [==============================] - 175s 2s/step - loss: 0.3695 - prob_loss: 0.2952 - dist_loss: 1.4876 - prob_kld: 0.0048 - dist_relevant_mae: 1.4862 - dist_relevant_mse: 10.7554 - val_loss: 0.3868 - val_prob_loss: 0.3035 - val_dist_loss: 1.6653 - val_prob_kld: 0.0061 - val_dist_relevant_mae: 1.6638 - val_dist_relevant_mse: 11.8497
Epoch 200/200
100/100 [==============================] - 175s 2s/step - loss: 0.3677 - prob_loss: 0.2947 - dist_loss: 1.4612 - prob_kld: 0.0036 - dist_relevant_mae: 1.4598 - dist_relevant_mse: 11.0554 - val_loss: 0.3881 - val_prob_loss: 0.3039 - val_dist_loss: 1.6848 - val_prob_kld: 0.0065 - val_dist_relevant_mae: 1.6834 - val_dist_relevant_mse: 12.2113

Loading network weights from 'weights_last.h5'.


NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:07<00:02,  1.89it/s, 0.668 -> 0.839]


Using optimized values: prob_thresh=0.665958, nms_thresh=0.3.
Saving to 'thresholds.json'.
Training UNET model
number of training images:	 173
number of validation images:	 19
image size (2D):		 (256, 256)
axes:				 SYXC
channels in / out:		 1 / 1
Config(axes='YXC', n_channel_in=1, n_channel_out=1, n_dim=2, probabilistic=False, train_batch_size=1, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_epochs=200, train_learning_rate=0.0001, train_loss='mae', train_reduce_lr={'patience': 5, 'factor': 0.5}, train_steps_per_epoch=400, train_tensorboard=True, unet_input_shape=(None, None, 1), unet_kern_size=3, unet_last_activation='linear', unet_n_depth=4, unet_n_first=48, unet_residual=True)
Epoch 1/200
400/400 [==============================] - 10s 24ms/step - loss: 0.1830 - mse: 0.1120 - mae: 0.1830 - val_loss: 0.1036 - val_mse: 0.0695 - val_mae: 0.1036
Epoch 2/200
400/400 [==============================] - 8s 19ms/ste